In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words= set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Task 1: (15 pts.)  
Without using anything but the raw json data (i.e., via the variable "data", and not the pandas dataframe "A"), write code to find the counts of all unique question categories. Your answer should match the above.

In [2]:
#Q1
import json
with open('/content/drive/My Drive/NLP/exam/JEOPARDY_QUESTIONS1.json', 'r') as f:      
    rawj = json.load(f) 
data = pd.DataFrame(rawj)  

from collections import Counter  
ct=Counter([x['category'] for x in rawj if x.get('category')])#\'category\':.*?(?=,)
ct=ct.most_common(len(ct))
ct[0:20]#for easy read only output 20

[('BEFORE & AFTER', 547),
 ('SCIENCE', 519),
 ('LITERATURE', 496),
 ('AMERICAN HISTORY', 418),
 ('POTPOURRI', 401),
 ('WORLD HISTORY', 377),
 ('WORD ORIGINS', 371),
 ('COLLEGES & UNIVERSITIES', 351),
 ('HISTORY', 349),
 ('SPORTS', 342),
 ('U.S. CITIES', 339),
 ('WORLD GEOGRAPHY', 338),
 ('BODIES OF WATER', 327),
 ('ANIMALS', 324),
 ('STATE CAPITALS', 314),
 ('BUSINESS & INDUSTRY', 311),
 ('ISLANDS', 301),
 ('WORLD CAPITALS', 300),
 ('U.S. GEOGRAPHY', 299),
 ('RELIGION', 297)]

Task 2: (30 pts.)  
Using the pandas dataframe "A", write code to see if there is a correlation between question length (in character counts) and the dollar value of the question. Hint: use the Pearson correlation via scipy.stats:

In [3]:
#Q2
data1=data
data1['value1'] = data['value'].str.extract('\$(\d+)', expand=False).astype(float)#clean digital
data1["question_clean"]=data1["question"].str.replace(r'[^\w\s]+', '')#remove Punctuation
data1["length"]= data1["question_clean"].str.len() 
data1=data1.fillna(0)

from scipy.stats import pearsonr
x = np.asarray(data1['value1'])
y = np.asarray(data1['length'])
print(pearsonr(x,y))

(0.10364886009811222, 0.0)


r=0.1112,p-value=0, p value is very small, we may reject H0 there a liitle connection between character len and value

Task 3: (20 pts.)
Write code to count the number of adjectives that begin with a vowel contained in all questions asked in 2006.

In [0]:
#q3
def getlen(x):
  x=x.lower()
  tags = set(['JJ', 'JJR', 'JJS'])
  x=word_tokenize(x)
  #print(x)
  pos_tags =nltk.pos_tag(x)
  #print(pos_tags)
  ret = []
  for word,pos in pos_tags:
    if (pos in tags and re.match("^[a|e|i|o|u]",word)):
      ret.append(word)
  #print(ret)
  return(len(ret))
data2006=data.loc[(data['air_date'].str.startswith('2006'))].reset_index(drop=True)
data2006["adj_vow"]=data2006["question"].apply(lambda x :getlen(x))


In [5]:
total=sum(data2006["adj_vow"])
total

3860

Task 4: (35 pts.)  
Train two binary classification models to predict whether or not a question is either a LITERATURE or SCIENCE question, using the k-nearest neighbor classifier and the ridge classifier.

Report out-of-sample accuracy, recall, precision, and f1 scores of both models. 

In [0]:
#Q4
knntest=data[['question','category']]

#may not use
def clean_text(croup):
  x=word_tokenize(croup)
  clean_senteces=pd.Series(x).str.replace("[^a-zA-Z0-9]"," ")
  clean_senteces=[s.lower() for s in clean_senteces]
  text_no_sp=[]
  for r in clean_senteces:
    tmp=r.split()
    sen_new=" ".join([i for i in tmp if i not in stop_words ])
    #sen_new=" ".join([i for i in tmp])
    if len(sen_new)>0:
      text_no_sp.append(sen_new)
  text_no_sp=" ".join(text_no_sp)
  return(text_no_sp)
  #return (clean_senteces)


In [7]:
#knntest["question1"]=knntest["question"].apply(lambda x :clean_text(x))
#knntest=knntest.loc[knntest['category']=="SCIENCE"].reset_index(drop=True)
knn=knntest.loc[knntest['category'].isin(['SCIENCE','LITERATURE'])  ].reset_index(drop=True)
knntest_label=np.asarray(knn["category"])
knn["question_cl"]=knn["question"].apply(lambda x :clean_text(x))
len(knn)

1015

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words=set(stop_words))
knntest_set = v.fit_transform(knn["question_cl"])

In [0]:
from sklearn import neighbors     
knn = neighbors.KNeighborsClassifier(n_neighbors=3, algorithm='auto', weights='distance', n_jobs=1)
knn.fit(knntest_set[0:900],knntest_label[0:900])
pred_label=knn.predict(knntest_set[900:1000])
insamp=knn.predict(knntest_set[0:900])

In [10]:
from sklearn import metrics
def metrics_result(actual, predict):
  print('precision:{0:.3f}'.format(metrics.precision_score(actual, predict, average='weighted')))
  print('recall:{0:0.3f}'.format(metrics.recall_score(actual, predict, average='weighted')))
  print('f1-score:{0:.3f}'.format(metrics.f1_score(actual, predict, average='weighted')))
print("train:0~900,test:900~1000")
print("=======out of sample========")
metrics_result(knntest_label[900:1000], pred_label)
print("==========in sample==========")
metrics_result(knntest_label[0:900], insamp)

train:0~900,test:900~1000
=======out of sample========
precision:0.890
recall:0.890
f1-score:0.890
==========in sample==========
precision:1.000
recall:1.000
f1-score:1.000


In [0]:
rilabel=[]
for i in knntest_label:
  if i=="SCIENCE":
    rilabel.append(0)#sci to 0
  else:
    rilabel.append(1)
from sklearn.linear_model import RidgeClassifier
clf =RidgeClassifier()
clf.fit(knntest_set[0:900],rilabel[0:900]) 
clfpre=clf.predict(knntest_set[900:1000])
incl=clf.predict(knntest_set[0:900])

In [12]:
print("train:0~900,test:900~1000")
print("=======out of sample========")
metrics_result(rilabel[900:1000], clfpre)
print("==========in sample==========")
metrics_result(rilabel[0:900], incl)


train:0~900,test:900~1000
=======out of sample========
precision:0.961
recall:0.960
f1-score:0.960
==========in sample==========
precision:1.000
recall:1.000
f1-score:1.000
